# Matches

In [5]:
# Setup ipython environment
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib.pyplot import *
from numpy import *
from positive import *
from nrutils import scsearch, gwylm
from pwca import *
from glob import glob
import lalsimulation as lalsim
from nrutils.analyze.match import match as match_object

(pwca.core)>> Catalog of calibration runs stored to "pwca.pwca_catalog"
(pwca.core)>> Metadata dictionary for calibration runs stored to "pwca.metadata_dict"
(pwca.core)>> Metadata dictionary for Ed's catalog paper stored to "pwca.catalog_paper_metadata"
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:

#
package_dir = parent( pwca.__path__[0] )
datadir = package_dir + 'data/version2/'
files = glob( datadir+'q*.txt' )

#
fref = 20

#
q_list = []
for a in pwca_catalog:
    q_list.append( int( round(a.m1/a.m2,2) ) )
mask = argsort(q_list)
pwca_catalog = array(pwca_catalog)[mask]

match_list = []
mod_phenomd_match_list = []
kk = 0
for a in pwca_catalog:
    
    #
    file_ = [ k for k in files if a.simname in k ][0]
    
    #
    Msol = 100.0
    Dmpc = 1000.0
    #
    theta = 0
    phi   = 0
    #
    fmin = 30.0
    fmax = 300.0
    
    # Load data for this case
    raw_data = loadtxt(file_)
    f,amp_td,amp_fd,dphi_td,dphi_fd,phi_td,phi_fd = raw_data.T
    # NOTE that the data used here must be consistent with the data used for calibration
    hcode_ = amp_fd * exp( 1j * phi_fd ) * sYlm(-2,2,2,theta,phi)
    hcode = array([f,hcode_*0.5,hcode_*0.5*1j]).T

    # Generate NR FD waveform array
    # ------------------------------------------------ #
    mask = f>0
    f     = f[mask]
    hcode = hcode[mask,:]
    hphys = physhf(hcode,Msol,Dmpc)

    # Generate MODEL FD waveform array
    # ------------------------------------------------ #
    
    if linalg.norm(a.X1)<1e-3:
        m1,m2 = a.m2,a.m1
        X1,X2 = a.X2,a.X1
        a.m1,a.m2 = m1,m2
        a.X1,a.X2 = X1,X2
    
    # External 2-spin mapping
    chi_p    = calc_chi_p(a.m1,a.X1,a.m2,a.X2,a.L)
    a1,model_theta = calc_effective_a1_theta( a.m1, a.m2, a.X1, a.X2, a.L )
    # Find index location of metadata for simname 
    k = [ z.split('.')[0] for z in pwca.metadata_dict['simname'] ].index(a.simname.split('.')[0])
    model_theta,m1,m2,eta,delta,chi_eff,chi_p,chi1,chi2,a1,a2 = pwca.metadata_dict['array_data'][k]
    
    # Generate a coprecessing waveform as 1D array (NEW INPUT STRUCTURE)
    hmodel = generate_pwca_waveform_helper( f, model_theta, eta, a1, chi1,chi2, chi_p, fref=fref ) * sYlm(-2,2,2,theta,phi)
    
    # Generate a coprecessing waveform as 1D array
    #hmodel = generate_pwca_waveform( f, a.m1, a.m2, a.X1, a.X2, a.L ) * sYlm(-2,2,2,theta,phi)
    #print sum(abs(hmodel-hmodel_old))/sum(abs(hmodel_old))
    
    hmodel_p = 0.5*(hmodel)
    hmodel_c = -1j*0.5*(hmodel)
    hcode_model = array([f,hmodel_p,hmodel_c]).T
    hphys_model = physhf( hcode_model,Msol,Dmpc )
    
    #
    mod_phenomd = generate_modified_phenomd(f, a.m1, a.m2, a.X1, a.X2, a.L)
    mod_phenomd_p = 0.5*(mod_phenomd)
    mod_phenomd_c = -1j*0.5*(mod_phenomd)
    mod_phenomd_code = array([f,mod_phenomd_p,mod_phenomd_c]).T
    mod_phenomd_phys = physhf( mod_phenomd_code,Msol,Dmpc )

    #
    phys_signal   = hphys
    phys_template = hphys_model

    # Create match object
    mo = match_object( phys_template,    phys_signal, fmin=fmin, fmax=fmax, signal_polarization=0, positive_f=True, psd_thing = lalsim.SimNoisePSDaLIGOZeroDetHighPower )
    moo= match_object( mod_phenomd_phys, phys_signal, fmin=fmin, fmax=fmax, signal_polarization=0, positive_f=True, psd_thing = lalsim.SimNoisePSDaLIGOZeroDetHighPower )

    mx  = mo.calc_template_pol_optimized_match()
    mxd = moo.calc_template_pol_optimized_match()
    print a.simname.replace('_','')[:16],'\t\t\t',1-mx,'\t\t\t',1-mxd,'\t\t\t',mxd>mx
    match_list.append(mx)
    mod_phenomd_match_list.append(mxd)

q1a04t120dP0T965 			0.0002022147746949532 			0.0007065727691145662 			False
q1a08t90dPm1T964 			0.0006094740648509012 			0.001063745879108069 			False
q1a08t30dPm25T96 			0.00017794639662815914 			0.0005772589231509695 			False
q1a04t60dPm1T965 			0.00018338844692789014 			0.00023834470402239738 			False
q1a08t60dPm1.5T9 			0.0002792873754692282 			0.0012491365100325558 			False
q1a08t120dP0T964 			0.00046465142817142535 			0.0010699151195415757 			False
q1a08t150dP0T964 			0.00016803569086143444 			0.00029262879030578226 			False
q1a04t30dPm2T965 			0.00014337763426852668 			0.00044030495062064023 			False
q1a04t150dP0T965 			0.00014805101412207478 			0.00017930668603338695 			False
q1a04t90dP0T9655 			0.00019883930948938477 			0.0002977672736211856 			False
q2a10a28ph0th150 			9.280499720287771e-05 			0.0003516269282636264 			False
q2a10a28ph0th120 			0.00027471931835332253 			0.0009391220581458359 			False
q2a10a28ph0th90 			0.0005969620338460713 			0.0026810068616272 			False
q2a10

In [12]:

figure(figsize=2*figaspect(0.618))

mask = argsort(match_list)

clr=rgb(4,jet=True)

q_set = [1,2,4,8][::-1]
passed_q = []
for k,a in enumerate(pwca_catalog[mask]):

    if mod_phenomd_match_list[ mask[k] ] > match_list[ mask[k] ]:
        plot( k+1, 1-match_list[ mask[k] ], mew=0, ms=35, ls='none', marker='o', color='yellow', alpha=0.3 )
        
    q = int( round(a.m1/a.m2,2) )
    plot( k+1, 1-mod_phenomd_match_list[ mask[k] ], mew=0, ms=6, ls='none', marker='o', color=clr[q_set.index(q)], label= 'Modified PhenomD' if not k else None )
    plot( k+1, 1-match_list[ mask[k] ], mew=2, ms=12, ls='none', marker='o', mfc='none', color=clr[q_set.index(q)], label= '$q=%i$'%q if not (q in passed_q) else None )
    plot( [k+1,k+1],[1-mod_phenomd_match_list[ mask[k] ],1-match_list[ mask[k] ]], color=clr[q_set.index(q)], alpha=0.5 )
    passed_q.append(q)
    
    #axhline(1e-2,ls='--',color='m')
    yscale('log')
    xlabel('Simulation ID')
    ylabel(r'$1-\langle \mathrm{NR} | \mathrm{Model} \rangle$')
    grid(True,axis='y')
    
    legend(ncol=3,frameon=True)
    
#
savefig('/Users/book/KOALA/puck/ll/data/version2/matches_diagnostic.pdf')

In [13]:
fig = mo.plot()
title('%s: match$=%f$'%(a.simname,mx));